Task 2 (Analytical). Ви - аналітик в стартапі, вам потрібно допомогти
маркетинговій команді розрахувати ключові показники. Ваш стартап - додаток
у якого модель монетизації є підписка. Спочатку юзер бере тріал (він триває 7
днів), після кінця тріалу він може відписатись, або ж продовжити платити за
тижневу підписку. Підписка коштує 9.99 за тиждень. Юзер може відписатись у
будь-який момент, після цього він не буде більше платити.

Ваше завдання - розрахувати LTV за допомогою Python. Будемо оцінювати
ваше загальне розуміння завдання та результат. У файлі data_analytics.csv ви
знайдете фрагмент бази даних, що вам потрібна, де:

Event Date – дата коли відбулась подія (тріал чи продовження підписки).

Subscription Offer Type – тип підписки (якщо вказано Free Trial - значить це
початок тріалу, якщо поле пусте - це підписка).

LTV – lifetime value. У цьому випадку рахується через конверсії з тріалу в
оплату і потім в наступні оплати. 

LTV = ([1]+[2]+[3]+[4]+[5]), де [1] = (dev proceeds * Conversion to trial), а [2] = [1] * 1st
purchase to 2nd, [3] = [2] * 2nd to 3rd і тд.

Важливо! Apple Store забирає собі 30% від вартості підписки. LTV в цьому
випадку ми будемо рахувати на 6 тижнів(1 тиждень тріалу та 5 підписки).

Importing lib


In [1]:
import pandas as pd

Looking at data

In [2]:
raw_data = pd.read_csv("data_analytics.csv", parse_dates=["Event Date"])

In [3]:
raw_data.head(3)

,Event Date,App Apple ID,Subscription Name,Standard Subscription Duration,Promotional Offer Name,Promotional Offer ID,Subscription Offer Type,Subscription Offer Duration,Marketing Opt-In Duration,Customer Currency,Proceeds Currency,Device,Country,Subscriber ID,Subscriber ID Reset,Refund,Purchase Date,Units
0,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,IDR,IDR,iPhone,ID,223411322456765433,NaN,NaN,NaN,1
1,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,USD,USD,iPhone,US,223411322456765434,NaN,NaN,NaN,1
2,2019-07-15,1,weekly 9.99 + 7D trial,7 Days,NaN,NaN,Free Trial,7 Days,NaN,INR,INR,iPhone,IN,223411322456765435,NaN,NaN,NaN,1


In [4]:
raw_data.shape

(2380, 18)

In [5]:
raw_data.nunique()

Event Date                          32
App Apple ID                         1
Subscription Name                    1
Standard Subscription Duration       1
Promotional Offer Name               0
Promotional Offer ID                 0
Subscription Offer Type              1
Subscription Offer Duration          1
Marketing Opt-In Duration            0
Customer Currency                   34
Proceeds Currency                   34
Device                               2
Country                             65
Subscriber ID                     1019
Subscriber ID Reset                  1
Refund                               0
Purchase Date                        0
Units                                1
dtype: int64

In [6]:
raw_data.isna().sum()

Event Date                           0
App Apple ID                         0
Subscription Name                    0
Standard Subscription Duration       0
Promotional Offer Name            2380
Promotional Offer ID              2380
Subscription Offer Type           1361
Subscription Offer Duration       1361
Marketing Opt-In Duration         2380
Customer Currency                    0
Proceeds Currency                    0
Device                               0
Country                              7
Subscriber ID                        0
Subscriber ID Reset               2376
Refund                            2380
Purchase Date                     2380
Units                                0
dtype: int64

Selecting relevant columns and changing names to convinient use 

In [7]:
df = raw_data[["Event Date", "Subscriber ID", "Subscription Offer Type"]].rename(columns={
    "Event Date": "date",
    "Subscriber ID": "id",
    "Subscription Offer Type": "paid"
})
df.paid = df.paid.replace("Free Trial", 0).fillna(1).apply(int)

In [8]:
df.head(3)

,date,id,paid
0,2019-07-15,223411322456765433,0
1,2019-07-15,223411322456765434,0
2,2019-07-15,223411322456765435,0


Looking at first and last date

In [9]:
print(df.date.min())
print(df.date.max())

2019-07-15 00:00:00
2019-08-24 00:00:00


Adding new features - week count, and subscribes count

In [10]:
df["week"] = df.date.apply(lambda x: ((x - df.date.min()).days // 7) + 1)

In [11]:
df.head(3)

,date,id,paid,week
0,2019-07-15,223411322456765433,0,1
1,2019-07-15,223411322456765434,0,1
2,2019-07-15,223411322456765435,0,1


In [12]:
count_of_subs = pd.DataFrame(df.id.value_counts() - 1).reset_index().rename(columns={"index":"id", "id":"number_of_subs"})

In [13]:
df = df.merge(count_of_subs, "left", on="id")

In [14]:
df.head(3)

,date,id,paid,week,number_of_subs
0,2019-07-15,223411322456765433,0,1,5
1,2019-07-15,223411322456765434,0,1,5
2,2019-07-15,223411322456765435,0,1,5


Let`s calculate LTV by user, and then calculate the mean


In [15]:
dev_proceeds = 9.99 * 2/3 # because of Apple store

In [16]:
ltv_array = count_of_subs.number_of_subs * dev_proceeds

In [17]:
customer_ltv = pd.DataFrame({"id":count_of_subs.id, "ltv":ltv_array})

In [18]:
customer_ltv.ltv.mean()

8.895250245338481

Lets calculate the LTV by conversion

In [19]:
# conversion - customers that have bought subscription devided by number of all customers
Conversion_from_trial = sum(count_of_subs.number_of_subs > 0) / len(count_of_subs)
# number of first purchase to second - number of customers with second subscription 
# devided by number of all customers with second
conv_to_2 = sum(count_of_subs.number_of_subs > 1) / sum(count_of_subs.number_of_subs > 0)
conv_to_3 = sum(count_of_subs.number_of_subs > 2) / sum(count_of_subs.number_of_subs > 1)
conv_to_4 = sum(count_of_subs.number_of_subs > 3) / sum(count_of_subs.number_of_subs > 2)
conv_to_5 = sum(count_of_subs.number_of_subs > 4) / sum(count_of_subs.number_of_subs > 3)

In [20]:
# calculating all components
first = dev_proceeds * Conversion_from_trial
second = first * conv_to_2
third = second * conv_to_3
forth = third * conv_to_4
fifth = forth * conv_to_5

In [21]:
ltv = first + second + third + forth + fifth

In [22]:
ltv

8.895250245338566